In [2]:
import datetime
import numpy as np
import pandas as pd
from scipy import sparse
from collections import Counter
import time
import os
import pickle

user_info = pd.read_csv('D:\\数据挖掘\data_format1\\user_info_format1.csv')
print(user_info.info())
print(user_info)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424170 entries, 0 to 424169
Data columns (total 3 columns):
user_id      424170 non-null int64
age_range    421953 non-null float64
gender       417734 non-null float64
dtypes: float64(2), int64(1)
memory usage: 9.7 MB
None
        user_id  age_range  gender
0        376517        6.0     1.0
1        234512        5.0     0.0
2        344532        5.0     0.0
3        186135        5.0     0.0
4         30230        5.0     0.0
...         ...        ...     ...
424165   395814        3.0     1.0
424166   245950        0.0     1.0
424167   208016        NaN     NaN
424168   272535        6.0     1.0
424169    18031        3.0     1.0

[424170 rows x 3 columns]


In [3]:
user_info.isnull().sum()

user_id         0
age_range    2217
gender       6436
dtype: int64

In [4]:
user_log = pd.read_csv('D:\\数据挖掘\data_format1\\user_log_format1.csv')
print(user_log.info())
print(user_log)
user_log.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54925330 entries, 0 to 54925329
Data columns (total 7 columns):
user_id        int64
item_id        int64
cat_id         int64
seller_id      int64
brand_id       float64
time_stamp     int64
action_type    int64
dtypes: float64(1), int64(6)
memory usage: 2.9 GB
None
          user_id  item_id  cat_id  seller_id  brand_id  time_stamp  \
0          328862   323294     833       2882    2661.0         829   
1          328862   844400    1271       2882    2661.0         829   
2          328862   575153    1271       2882    2661.0         829   
3          328862   996875    1271       2882    2661.0         829   
4          328862  1086186    1271       1253    1049.0         829   
...           ...      ...     ...        ...       ...         ...   
54925325   208016   107662     898       1346    7995.0        1110   
54925326   208016  1058313     898       1346    7995.0        1110   
54925327   208016   449814     898        9

user_id            0
item_id            0
cat_id             0
seller_id          0
brand_id       91015
time_stamp         0
action_type        0
dtype: int64

In [6]:
#去重（结果显示并无重复），检查是否有全为null的列并删去
user_info = user_info.drop_duplicates()
user_info = user_info.dropna(axis=0,how='all')
user_info

,user_id,age_range,gender
0,376517,6.0,1.0
1,234512,5.0,0.0
2,344532,5.0,0.0
3,186135,5.0,0.0
4,30230,5.0,0.0
...,...,...,...
424165,395814,3.0,1.0
424166,245950,0.0,1.0
424167,208016,NaN,NaN
424168,272535,6.0,1.0


In [7]:
#检查是否有全为null的行并删去
user_log = user_log.rename(columns={'seller_id': 'merchant_id'})
user_log = user_log.drop_duplicates(keep='first')#去除了13750198行
user_log = user_log.dropna(axis=0,how='all')
user_log

,user_id,item_id,cat_id,merchant_id,brand_id,time_stamp,action_type
0,328862,323294,833,2882,2661.0,829,0
1,328862,844400,1271,2882,2661.0,829,0
2,328862,575153,1271,2882,2661.0,829,0
3,328862,996875,1271,2882,2661.0,829,0
4,328862,1086186,1271,1253,1049.0,829,0
...,...,...,...,...,...,...,...
54925318,208016,572517,898,4992,6137.0,1110,0
54925324,208016,466472,898,1346,7995.0,1110,0
54925326,208016,1058313,898,1346,7995.0,1110,0
54925327,208016,449814,898,983,7995.0,1110,0


In [24]:
user_info.user_id.value_counts()#统计info有多少个用户

2047      1
398440    1
310333    1
304190    1
306239    1
         ..
354808    1
352761    1
358906    1
356859    1
2049      1
Name: user_id, Length: 424170, dtype: int64

In [25]:
user_log.user_id.value_counts()

254263    12159
276887     8448
109251     7519
23106      6127
179074     5883
          ...  
180864        2
69967         2
306187        2
127219        2
234543        2
Name: user_id, Length: 424170, dtype: int64

In [32]:
print(user_log.merchant_id.value_counts())
print(user_log.item_id.value_counts())
print(user_log.cat_id.value_counts())
print(user_log.brand_id.value_counts())

3760    605585
3828    595469
1102    417087
4173    412122
4044    387823
         ...  
3808       180
4103       157
4449       127
4215       123
2979       106
Name: merchant_id, Length: 4995, dtype: int64
631714     30567
67897      29011
783997     22310
61518      21230
1059899    14980
           ...  
250063         1
661706         1
577967         1
86172          1
907255         1
Name: item_id, Length: 1090390, dtype: int64
662     3591703
1505    1402663
737     1351591
389     1259809
656     1148067
         ...   
961           1
169           1
947           1
945           1
540           1
Name: cat_id, Length: 1658, dtype: int64
3738.0    622507
1446.0    594581
1214.0    416792
5376.0    411380
82.0      387251
           ...  
8140.0         1
5977.0         1
615.0          1
4628.0         1
3425.0         1
Name: brand_id, Length: 8443, dtype: int64


In [33]:
user_log.action_type.value_counts()

0    35080757
2     3013845
3     3005723
1       74807
Name: action_type, dtype: int64

In [8]:
user_log_1111 = user_log[user_log.time_stamp==1111]
user_log_1111

,user_id,item_id,cat_id,merchant_id,brand_id,time_stamp,action_type
171,328862,406349,1280,2700,5476.0,1111,0
173,328862,807126,1181,1963,6109.0,1111,0
174,328862,406349,1280,2700,5476.0,1111,2
428,234512,399860,962,305,6300.0,1111,0
429,234512,240182,81,3018,4144.0,1111,2
...,...,...,...,...,...,...,...
54925287,208016,76124,898,3716,7995.0,1111,0
54925288,208016,292204,955,2792,2431.0,1111,0
54925290,208016,514608,898,3716,7995.0,1111,0
54925292,208016,51712,898,3763,8434.0,1111,0


In [9]:
user_log_1111.action_type.value_counts()

0    5361364
2    1140970
3     156450
1      14276
Name: action_type, dtype: int64

In [10]:
user_log_1110 = user_log[user_log.time_stamp<1111]
#print(user_log_1110)
#min_date=min(user_log.time_stamp)
#min_date
user_log_1110.action_type.value_counts()/184


0    161518.195652
3     15485.179348
2     10178.668478
1       328.972826
Name: action_type, dtype: float64

In [11]:
#数据清洗、预处理
user_info.age_range.fillna(user_info.age_range.median(),inplace=True)#年龄用中位数填充
user_info.gender.fillna(user_info.gender.mode()[0],inplace=True)# 性别用众数填充
print('Check any missing value?\n',user_info.isnull().any())# 检查缺省值

Check any missing value?
 user_id      False
age_range    False
gender       False
dtype: bool


In [5]:
# brand_id使用所在seller_id对应的brand_id的众数填充
def get_Logs():
    '''
    :parameters: None: None
    :return: userLog: pd.Dataframe
    :Purpose: 
    方便与其他函数调取原始的行为数据，同时已对缺失省进行调整
    使用pickle模块进行序列话，加快速度读写
    '''
    filePath = 'D:\\数据挖掘\data_format1\\Logs.pkl'
    #global user_log
    if os.path.exists(filePath):
        user_log = pickle.load(open(filePath,'rb'))
    else:
        user_log = pd.read_csv('D:\\数据挖掘\data_format1\\user_log_format1.csv')
        print('Is null? \n',user_log.isnull().sum())
 
        ## 对brand_id缺失值进行处理
        missingIndex = user_log[user_log.brand_id.isnull()].index
        ## 思路：找到所有商店所拥有brand_id的众数，并对所缺失的brand_id与其相对应的商店进行填充
        sellerMode = user_log.groupby(['seller_id']).apply(lambda x:x.brand_id.mode()[0]).reset_index()
        pickUP = user_log.loc[missingIndex]
        pickUP = pd.merge(pickUP,sellerMode,how='left',on=['seller_id'])[0].astype('float32')
        pickUP.index = missingIndex
        user_log.loc[missingIndex,'brand_id'] = pickUP
        del pickUP,sellerMode,missingIndex
        print('--------------------')
        print('Is null? \n',user_log.isnull().sum())
        pickle.dump(user_log,open(filePath,'wb'))
    return user_log
user_log = get_Logs()

Is null? 
 user_id            0
item_id            0
cat_id             0
seller_id          0
brand_id       91015
time_stamp         0
action_type        0
dtype: int64
--------------------
Is null? 
 user_id        0
item_id        0
cat_id         0
seller_id      0
brand_id       0
time_stamp     0
action_type    0
dtype: int64


In [9]:
#特征提取
# 用户基本信息：年龄，性别（类别型特征）
df_age = pd.get_dummies(user_info.age_range,prefix='age')# 对age进行哑编码
df_sex = pd.get_dummies(user_info.gender)# 对gender进行哑编码并改变列名
df_sex.rename(columns={0:'female',1:'male',2:'unknown'},inplace=True)
user_info = pd.concat([user_info.user_id, df_age, df_sex], axis=1)# 整合user信息
del df_age,df_sex
print(user_info.info())
print(user_info)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424170 entries, 0 to 424169
Data columns (total 13 columns):
user_id    424170 non-null int64
age_0.0    424170 non-null uint8
age_1.0    424170 non-null uint8
age_2.0    424170 non-null uint8
age_3.0    424170 non-null uint8
age_4.0    424170 non-null uint8
age_5.0    424170 non-null uint8
age_6.0    424170 non-null uint8
age_7.0    424170 non-null uint8
age_8.0    424170 non-null uint8
female     424170 non-null uint8
male       424170 non-null uint8
unknown    424170 non-null uint8
dtypes: int64(1), uint8(12)
memory usage: 8.1 MB
None
        user_id  age_0.0  age_1.0  age_2.0  age_3.0  age_4.0  age_5.0  \
0        376517        0        0        0        0        0        0   
1        234512        0        0        0        0        0        1   
2        344532        0        0        0        0        0        1   
3        186135        0        0        0        0        0        1   
4         30230        0        0        

In [12]:
# 提取全部的原始行为数据...
totalActions = user_log[["user_id","action_type"]]
totalActions.head()
totalActions

,user_id,action_type
0,328862,0
1,328862,0
2,328862,0
3,328862,0
4,328862,0
...,...,...
54925325,208016,0
54925326,208016,0
54925327,208016,0
54925328,208016,0


In [13]:
# 对行为类别进行哑编码，0 表示点击， 1 表示加入购物车, 2 表示购买，3 表示收藏.
df = pd.get_dummies(totalActions['action_type'],prefix='userTotalAction')

# 统计日志行为中用户点击、加购、购买、收藏的总次数
totalActions = pd.concat([totalActions.user_id, df], axis=1).groupby(['user_id'], as_index=False).sum()
totalActions['userTotalAction'] = totalActions['userTotalAction_0']+totalActions['userTotalAction_1']+totalActions['userTotalAction_2']+totalActions['userTotalAction_3']
del df
totalActions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 424170 entries, 0 to 424169
Data columns (total 6 columns):
user_id              424170 non-null int64
userTotalAction_0    424170 non-null float64
userTotalAction_1    424170 non-null float64
userTotalAction_2    424170 non-null float64
userTotalAction_3    424170 non-null float64
userTotalAction      424170 non-null float64
dtypes: float64(5), int64(1)
memory usage: 22.7 MB


In [15]:
#用户交互次数在所有用户中的地位
print('所有用户交互次数：'+str(user_log.shape[0]))
print('所有用户数：'+str(user_log['user_id'].nunique()))
print('所有用户平均交互次数：'+str(user_log.shape[0]/user_log['user_id'].nunique()))
totalActions['userTotalActionRatio'] = totalActions['userTotalAction']/user_log.shape[0]#用户交互次数占所有交互次数的比例
totalActions['userTotalActionDiff'] = totalActions['userTotalAction']-user_log.shape[0]/user_log['user_id'].nunique()#用户交互次数与平均交互次数的差值

所有用户交互次数：54925330
所有用户数：424170
所有用户平均交互次数：129.48895490015795


In [16]:
#用户点击次数在所有用户点击次数中的地位
print('所有用户点击次数：'+str(user_log[user_log.action_type==0].shape[0]))
totalActions['userClickRatio'] = totalActions['userTotalAction_0']/user_log[user_log.action_type==0].shape[0]#用户点击次数占所有点击次数的比例
print('用户平均点击次数：'+str(user_log[user_log.action_type==0].shape[0]/user_log['user_id'].nunique()))
totalActions['userClickDiff'] = totalActions['userTotalAction_0']-user_log[user_log.action_type==0].shape[0]/user_log['user_id'].nunique()#用户点击次数与平均点击次数的差值

所有用户点击次数：48550713
用户平均点击次数：114.46050640073555


In [17]:
print(totalActions)

        user_id  userTotalAction_0  userTotalAction_1  userTotalAction_2  \
0             1               27.0                0.0                6.0   
1             2               47.0                0.0               14.0   
2             3               63.0                0.0                4.0   
3             4               49.0                0.0                1.0   
4             5              150.0                0.0               13.0   
...         ...                ...                ...                ...   
424165   424166               79.0                0.0               11.0   
424166   424167               28.0                0.0                6.0   
424167   424168              216.0                0.0                6.0   
424168   424169              277.0                0.0               17.0   
424169   424170               39.0                0.0                1.0   

        userTotalAction_3  userTotalAction  userTotalActionRatio  \
0                  